<a href="https://colab.research.google.com/github/chambai/Deep_Learning_Course/blob/main/Week%201%20DL%202/HyperparameterTuningIris_Sln1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hyperparameter Tuning Iris

Add hyperparemter tuning to the Iris DNN.
Adds hyperparameter tuning to the second layer of the Iris neural network.
Tries number of units from 10 to 200

In [ ]:
# install the Keras-tuner library
!pip install keras-tuner

In [ ]:
# import the library and refer to it as kt
import kerastuner as kt

In [ ]:
import pandas as pd
from IPython.display import display
from sklearn.datasets import load_iris
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt

# get the data
data = load_iris()
df = pd.DataFrame(data.data, columns=data.feature_names)

# Normalise the data
df_norm = normalize(df)

# split the data into train and test
x_train, x_test, y_train, y_test = train_test_split(df_norm, data.target, random_state=0)

# one-hot encode the data
y_train = utils.to_categorical(y_train)
y_test = utils.to_categorical(y_test)


In [ ]:
# build the model within a function
# Apply hyperparmeter tuning to the number of units in the second hidden layer in the model
def model_builder(hp):
  model = Sequential()
  model.add(tf.keras.layers.Dense(input_dim=4, units=10, activation='relu'))
  hp_units = hp.Int('units', min_value = 10, max_value = 200, step = 10)
  model.add(tf.keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(tf.keras.layers.Dense(units=3, activation='softmax'))
  #model.add(tf.keras.layers.Dense(units=hp.Int('units', min_value = 15, max_value = 350, step = 10), activation='softmax'))
  model.compile(optimizer = "adam", loss = 'categorical_crossentropy', metrics = ['accuracy'])
  return model

In [ ]:
# instatiate the tuner and perform hypertuning
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 125,
                     factor = 3,        # factor is a number that determines how many models are created to run in parallel whaen testing the hyperparameters
                     directory = 'my_dir',  # directory that the tuned hyperparameter results are stored in (change the name of this directory if you get the message INFO:tensorflow:Oracle triggered exit)
                     project_name = 'intro_to_kt') 

In [ ]:
# define a callback to clear the training outputs at the end of every training step
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

The hyperparameter search is performed using `tuner.search`.  The arguments are the same as for the fit method for the model.
When the search is complete, it returns the best hyperparameters.

**Note:** if you do not get a display of updating training epochs when running the next cell and you only see this message:   **INFO:tensorflow:Oracle triggered exit**, the keras tuner has not worked correctly.  To fix this, 
update `directory='my_dir'` (which is two code cells above) to have a different directory name. Keras stores the tuning results in this directory and can sometimes error if you are changing the model alot.

In [ ]:
import IPython
# run the hyperparameter search
tuner.search(x_train, y_train, epochs = 125, validation_data = (x_test, y_test), callbacks = [ClearTrainingOutput()])

In [ ]:
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]
print(best_hps.values)

In [ ]:
# apply the optimal hyperparameters to the model and print the summary
model = tuner.hypermodel.build(best_hps)
model.summary()

In [ ]:
# retrain the model with the optimum hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.summary()
history = model.fit(x_train, y_train, epochs = 125, validation_data = (x_test, y_test))

In [ ]:
# summarize the history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

# summarize the history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()